In [13]:
%load_ext autoreload
%autoreload 2

Config file:  base.json
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from actor_net import ActorNet
import numpy as np
from gameworlds.hex_world import HexState

Config file:  base.json


In [3]:
with open('data/2022-03-27T14-17-35_4x4.npy', 'rb') as f:
    states = np.load(f)
    y_train = np.load(f)
    y_train_value = np.load(f)

In [9]:
states.shape

(7348, 17)

In [5]:

x_train = np.array(list(map(HexState.from_array_to_vector, states)))

In [12]:
np.unique(states, axis=0).shape

(5016, 17)

In [8]:
states

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [-1., -0., -0., ..., -0., -0., -0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [-1., -0., -0., ..., -0.,  1.,  1.],
       [ 1., -1.,  0., ...,  0.,  0.,  0.],
       [-1.,  1.,  1., ..., -0., -0., -0.]])

In [5]:
anet = ActorNet()

In [6]:
anet.model.summary()

Model: "2HNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 7, 7, 9)]    0           []                               
                                                                                                  
 conv2d_14 (Conv2D)             (None, 3, 3, 64)     14464       ['Input[0][0]']                  
                                                                                                  
 leaky_re_lu_14 (LeakyReLU)     (None, 3, 3, 64)     0           ['conv2d_14[0][0]']              
                                                                                                  
 batch_normalization_14 (BatchN  (None, 3, 3, 64)    256         ['leaky_re_lu_14[0][0]']         
 ormalization)                                                                                

In [ ]:
scaled = y_train ** 5

In [ ]:
scaled = scaled / np.sum(scaled, axis=1).reshape(len(scaled), 1)

In [ ]:
anet.train(x_train=x_train,y_train=scaled, y_train_value=y_train_value, epochs=100)

In [ ]:
anet.save_params(201)